In [2]:
import pandas as pd

In [3]:
github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'
url = f'{github_url}?raw=1'
df = pd.read_csv(url)
df = df.iloc[:300]

In [4]:
from sentence_transformers import SentenceTransformer
model_name = 'multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
answer_llm = df.iloc[0].answer_llm

In [7]:
embedding_model.encode(answer_llm)

array([-4.22446579e-01, -2.24855691e-01, -3.24058741e-01, -2.84758866e-01,
        7.25708902e-03,  1.01186641e-01,  1.03716828e-01, -1.89983517e-01,
       -2.80596483e-02,  2.71588504e-01, -1.15337148e-01,  1.14666402e-01,
       -8.49587694e-02,  3.32365334e-01,  5.52724376e-02, -2.22195536e-01,
       -1.42540991e-01,  1.02519184e-01, -1.52333423e-01, -2.02912375e-01,
        1.98422819e-02,  8.38149786e-02, -5.68631887e-01,  2.32843235e-02,
       -1.67292640e-01, -2.39256412e-01, -8.05465057e-02,  2.57083327e-02,
       -8.15466121e-02, -7.39289969e-02, -2.61550128e-01,  1.92576181e-02,
        3.22909296e-01,  1.90357268e-01, -9.34726486e-05, -2.13165566e-01,
        2.88943667e-02, -1.79528147e-02, -5.92762418e-02,  1.99918032e-01,
       -4.75170687e-02,  1.71633765e-01, -2.45915428e-02, -9.38056633e-02,
       -3.57002944e-01,  1.33263484e-01,  1.94045842e-01, -1.18530914e-01,
        4.56915259e-01,  1.47728145e-01,  3.35945159e-01, -1.86959863e-01,
        2.45954141e-01, -

In [8]:
df.head()

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


In [12]:
def compute_similarity(row):
    answer_orig_vector = embedding_model.encode(row['answer_orig'])
    answer_llm_vector = embedding_model.encode(row['answer_llm'])
    return answer_llm_vector.dot(answer_orig_vector)

df['similarity'] = df.apply(lambda row: compute_similarity(row), axis=1)

In [13]:
df['similarity'].quantile(0.75)

31.67430353164673

In [18]:
import numpy as np

def compute_norm_vector(vector):
    norm = np.sqrt((vector * vector).sum())
    return vector / norm

def compute_cosine_similarity(row):
    answer_orig_vector = embedding_model.encode(row['answer_orig'])
    answer_llm_vector = embedding_model.encode(row['answer_llm'])

    answer_orig_norm_vector = compute_norm_vector(answer_orig_vector)
    answer_llm_norm_vector = compute_norm_vector(answer_llm_vector)

    return answer_orig_norm_vector.dot(answer_llm_norm_vector)

df['cosine_similarity'] = df.apply(lambda row: compute_cosine_similarity(row), axis=1)

In [19]:
df['cosine_similarity'].quantile(0.75)

0.8362347632646561

In [20]:
!pip install rouge


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [29]:
from rouge import Rouge
rouge_scorer = Rouge()

r = df.iloc[10]
scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

In [30]:
np.average([scores['rouge-1']['f'],scores['rouge-2']['f'],scores['rouge-l']['f']])

0.35490034990035496

In [31]:
def compute_rouge_similarity(row):
    scores = rouge_scorer.get_scores(row['answer_llm'], row['answer_orig'])[0]
    return pd.Series(
        {'rouge-1': scores['rouge-1']['f'], 
         'rouge-2': scores['rouge-2']['f'],
         'rouge-l': scores['rouge-l']['f']})

df = df.join(df.apply(compute_rouge_similarity, axis=1))


In [32]:
df.head()

,answer_llm,answer_orig,document,question,course,similarity,cosine_similarity,rouge-1,rouge-2,rouge-l
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp,17.515995,0.506754,0.095238,0.028169,0.095238
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp,13.418406,0.388549,0.125000,0.055556,0.093750
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp,25.313255,0.718599,0.415584,0.177778,0.389610
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp,12.147417,0.337266,0.216216,0.047059,0.189189
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp,18.747723,0.521792,0.142076,0.033898,0.120219


In [35]:
np.average(df['rouge-2'])

0.20696501983423318